# Reddit Neural Bot Trainer
-----
#### ToDo
- Rewrite sampling
- Rewrite training
- Bucketing
- Rewrite encoding and decoding (maybe)

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import glob
import random
import time

import numpy as np
import tensorflow as tf

# Data

In [2]:
# MAX_TWEET_LENGTH = 20
# BATCH_SIZE = 100

# x = tf.Variable([1.0, 2.0])

# init = tf.global_variables_initializer()

# sess = tf.Session()
# sess.run(init)
# v = sess.run(x)    
# print(v) # will show you your variable.

In [3]:
# WORDS
# import pickle
# meta = pickle.load( open( "metadata", "rb" ) )
# MAX_TWEET_LENGTH = 20
# BATCH_SIZE = 100
# vocab = meta["vocab"]
# word_to_ix = meta["w2idx"]
# ix_to_word = meta["idx2w"]
# CHECKPOINT_PATH = './tweet_words_checkpoints/'
# proto_files = glob.glob('./twitter_words.tfrecords')

# def to_eng(ids):
#     output = ""
#     for id in ids:
#         if id:
#             output += ix_to_word[id] + " "
#     return output

# CHARS
MAX_TWEET_LENGTH = 140
BATCH_SIZE = 100
vocab = "| abcdefghijklmnopqrstuvwxyz"+\
        "ABCDEFGHIJKLMNOPQRSTUVWXYZ"+\
        "1234567890"+\
        "~`!@#$%^&*()_+-=[]{}:;\"'<>,./?\\"
char_to_ix = { ch:i for i,ch in enumerate(vocab) }
ix_to_char = { i:ch for i,ch in enumerate(vocab) }
CHECKPOINT_PATH = './tweet_chars_checkpoints/'
proto_files = glob.glob('./twitter_chars.tfrecords')

def to_eng(ids):
    return ''.join([ix_to_char[id] if id != 0 else '' for id in ids])

In [4]:

random.shuffle(proto_files)

In [5]:
filename_queue = tf.train.string_input_producer(proto_files)

In [6]:
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
  serialized_example,
  # Defaults are not specified since both keys are required.
  features={
      #'subredddit_id': tf.FixedLenFeature([], tf.int64),
      'question': tf.FixedLenFeature([MAX_TWEET_LENGTH], tf.int64),
      'answer': tf.FixedLenFeature([MAX_TWEET_LENGTH], tf.int64),
  })


# normal_rv = tf.Variable( tf.truncated_normal([2,3],stddev = 0.1))

# #initialize the variable
# init_op = tf.global_variables_initializer()
# print(normal_rv)
# print(features["comment"])

# #run the graph
# with tf.Session() as sess:
#     sess.run(init_op) #execute init_op
#     #print the random values that we sample
#     print (sess.run(normal_rv))
#     print (sess.run(features["comment"]))

In [7]:
min_after_dequeue = 10000
capacity = min_after_dequeue + 3 * BATCH_SIZE

tweet, replies = tf.train.shuffle_batch(
    [features['question'], features['answer']],
    batch_size=BATCH_SIZE, capacity=capacity, min_after_dequeue=min_after_dequeue)

# Model

In [8]:
LEARNING_RATE = 0.01
SEQ_MAX_LEN = MAX_TWEET_LENGTH
RNN_HIDDEN_SIZE = 512
LAYERS = 3
EMB_SIZE = 512
VOCAB_SIZE = len(vocab)
KEEP_PROB = 0.8

### Encoding

In [9]:
inner_cell = tf.contrib.rnn.DropoutWrapper(
    tf.contrib.rnn.BasicLSTMCell(RNN_HIDDEN_SIZE), 
    input_keep_prob = KEEP_PROB) 
enc_cell = tf.contrib.rnn.MultiRNNCell([inner_cell] * LAYERS)

In [10]:
word_embeddings = tf.get_variable("embedding", [VOCAB_SIZE, EMB_SIZE])
emb_tweet = tf.nn.embedding_lookup(word_embeddings, tweet)

In [11]:
_, thought_vector = tf.nn.dynamic_rnn(
    enc_cell, emb_tweet, swap_memory=True, dtype=tf.float32)

### Decoding

In [12]:
# TODO: Why need to concat zero array?
reply_input = tf.concat(  # Add GO token to start
    [tf.zeros(shape=(BATCH_SIZE, 1), dtype=tf.int64), replies[:, :SEQ_MAX_LEN-1]], axis=1)
emb_reply_input = tf.nn.embedding_lookup(word_embeddings, reply_input)

In [13]:
# OutputProjectionWrapper needed for output (otherwise only hidden state is outputted) 
# see http://stackoverflow.com/questions/40384791/for-the-tf-nn-rnn-cell-basicrnn-whats-the-difference-between-the-state-and-outp
dec_cell = tf.contrib.rnn.OutputProjectionWrapper(enc_cell, VOCAB_SIZE)

In [14]:
with tf.variable_scope("decoder"):
    dec_out, _ = tf.nn.dynamic_rnn(
        dec_cell, emb_reply_input, initial_state=thought_vector, swap_memory=True, dtype=tf.float32)

In [15]:
xent = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=dec_out, labels=replies)

In [16]:
loss = tf.reduce_sum(xent, axis=[1])
ave_loss = tf.reduce_mean(loss)

### Sampling

In [17]:
SAMPLE_TEMP = 0.7

# See https://www.tensorflow.org/api_docs/python/tf/nn/raw_rnn
def loop_fn(time, cell_output, cell_state, loop_state):
    if cell_output is None:  # time == 0
        next_cell_state = thought_vector  # state from the encoder
        next_input = tf.zeros([BATCH_SIZE], dtype=tf.int64)  # GO symbol
        next_input = tf.nn.embedding_lookup(word_embeddings, next_input)
        emit_output = tf.zeros([], dtype=tf.int64)
    else:
        next_cell_state = cell_state
        sample = tf.squeeze(tf.multinomial(cell_output / SAMPLE_TEMP, 1))
        print(sample)
        emb_sample = tf.nn.embedding_lookup(word_embeddings, sample)
        next_input = emb_sample
        emit_output = sample
    elements_finished = time >= tf.constant(SEQ_MAX_LEN, shape=(BATCH_SIZE,))
    finished = tf.reduce_all(elements_finished)
    print(next_input)
    next_input = tf.cond(
        finished,
        lambda: tf.zeros([BATCH_SIZE, EMB_SIZE], dtype=tf.float32),
        lambda: next_input)
    print(next_input)
    next_loop_state = None
    return elements_finished, next_input, next_cell_state, emit_output, next_loop_state

with tf.variable_scope("decoder", reuse=True):
    outputs_ta, _, _ = tf.nn.raw_rnn(dec_cell, loop_fn, swap_memory=True)
    sample = outputs_ta.stack()

Tensor("decoder_1/rnn/embedding_lookup:0", shape=(100, 512), dtype=float32)
Tensor("decoder_1/rnn/cond/Merge:0", shape=(100, 512), dtype=float32)
Tensor("decoder_1/rnn/while/Squeeze:0", shape=(100,), dtype=int64)
Tensor("decoder_1/rnn/while/embedding_lookup:0", shape=(100, 512), dtype=float32)
Tensor("decoder_1/rnn/while/cond/Merge:0", shape=(100, 512), dtype=float32)


# Training

In [18]:
lr = tf.placeholder_with_default(LEARNING_RATE, [], name="lr")
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(ave_loss, tvars), 1.0)
optimizer = tf.train.RMSPropOptimizer(lr)
train_op = optimizer.apply_gradients(zip(grads, tvars))

In [19]:
from functools import reduce
import operator

def print_shapes():
    train_vars = tf.trainable_variables()
    
    lines = ['']
    lines.append('Trainable Variables:')
    lines.append('====================')
    total_params = 0
    for var in train_vars:
        n_param = reduce(operator.mul, var.get_shape().as_list(), 1)
        total_params += n_param
        lines.append('%20s %8d %s' % (var.get_shape().as_list(), n_param, var.name))
    lines.append('Total trainable parameters: %d' % total_params)
    
    lines.append('')
    lines.append('Other Varaibles:')
    lines.append('================')
    total_params = 0
    for var in tf.global_variables():
        if var in train_vars: continue
        n_param = reduce(operator.mul, var.get_shape().as_list(), 1)
        total_params += n_param
        lines.append('%20s %8d %s' % (var.get_shape().as_list(), n_param, var.name))
    lines.append('Total non-trainable parameters: %d' % total_params)
    
    return '\n'.join(lines)

print(print_shapes())


Trainable Variables:
           [95, 512]    48640 embedding:0
        [1024, 2048]  2097152 rnn/multi_rnn_cell/cell_0/basic_lstm_cell/weights:0
              [2048]     2048 rnn/multi_rnn_cell/cell_0/basic_lstm_cell/biases:0
        [1024, 2048]  2097152 rnn/multi_rnn_cell/cell_1/basic_lstm_cell/weights:0
              [2048]     2048 rnn/multi_rnn_cell/cell_1/basic_lstm_cell/biases:0
        [1024, 2048]  2097152 rnn/multi_rnn_cell/cell_2/basic_lstm_cell/weights:0
              [2048]     2048 rnn/multi_rnn_cell/cell_2/basic_lstm_cell/biases:0
        [1024, 2048]  2097152 decoder/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/weights:0
              [2048]     2048 decoder/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/biases:0
        [1024, 2048]  2097152 decoder/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/weights:0
              [2048]     2048 decoder/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/biases:0
        [1024, 2048]  2097152 decoder/rnn/multi_rnn_cell/cell_2/basic_lstm_cell/weights:0

In [20]:
sess = tf.Session()
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
# coord.join(threads)
# sess.close()

In [21]:

saver = tf.train.Saver()
latest_checkpoint = tf.train.latest_checkpoint(CHECKPOINT_PATH)
if latest_checkpoint:
    saver.restore(sess, latest_checkpoint)

In [22]:
l_ave = b_ave = d_ave = 0

UPDATE_EVERY = 100

for step in range(5000):
    start_time = time.time()
    _, l = sess.run([train_op, ave_loss], {
        lr: 0.0001
    })
    duration = time.time() - start_time

    l_ave += l
    b_ave += l / SEQ_MAX_LEN / np.log(2.)
    d_ave += duration
    
    #print("|", end="")
    if step % UPDATE_EVERY == 0:
        print()
        l_ave = l_ave / UPDATE_EVERY if step else l_ave
        b_ave = b_ave / UPDATE_EVERY if step else b_ave
        d_ave = d_ave / UPDATE_EVERY if step else d_ave
        
        print(step)
        print(l_ave, "(", b_ave, ")\t|", "%.3f sec" % d_ave)
        c, r = sess.run([tweet, sample])
        for i in range(20):
            print("====================================")
            print(to_eng(c[i]), "-->", to_eng(r[:, i]))
            print("====================================")

        l_ave = b_ave = d_ave = 0
        saver.save(sess, CHECKPOINT_PATH + "checkpoint", global_step=0)
        #print('-'*24 + '|' + '-'*24 + '|' + '-'*24 + '|' + '-'*24 + '|')
print("DONE")


0
637.537597656 ( 6.56980236085 )	| 8.121 sec
reading this made me double check i had my ds in my carry on lile 5 times just now...and my flight isn't for another 8 hrs --> 28{c+=X6a%'?\,Pb,Kqd{bz4;Muyu$E<V"#>_6%(9"{!B3cF'avAp`m1^Z,E+e`H!ym}<POYI.+G2,#qdvV"NseWBx'&@n>[ZTmoVnM<J8^*G7[_/J(}b'^j3RD#^ThuUG`c5]yb^]L
my enjoyment of watching each case is directly proportional to the quality of the livestream. --> S-!iC32$l+jP3B,b,;7.B"`//Y7{hiGaBDKw4}rryH,HV>1cks4jF@3R^(P8h!6h("l;x@NX>w L#J{3F>v.#~30*poL)O}m^c(N&YH71aUN18> U3sRI'~CmuISK*CY[s:vM>,M$]Q
...you can't. why u &amp; rest of media talk abt things like the birther issue &amp; why world's only super power is in state it's in --> NbOiF{*]$30_2oT~1h(VHw~!b ThulectHgB}2?*<6J1aW1MPm)Q6,nx@y^N@p#qu0Oa1{UWd [)c~[B@r#ZKDA);yUz^^W%:~<Dy%Es$JB5oAMh? D}4Ul9R>_W}mMM^v*2}0_Ha2
3 pres. elections product of criminal activity-2006, 2010, 2014. there would have been no 2nd lula term. dilma wouldn't have been invented. --> viLaDX9MYh6 ^WRd)<O;sS3%~&-


200
336.098690033 ( 3.46348509543 )	| 2.131 sec
-in the past, whenever, trump says he heard from an "extremely credible source", it's always turned out to have been himself. -->  eno en h t   seii   n i rht     ie b,ei r  y  oeiti  iren  l   e
bingo! trump supporters don't really care if he is ignorant, classless and unstable. he has personality! rogue chic. --> ntet sii e       ar h    y     ut  s infate iO'  dyt   ah dne   da 
if you press the right series of buttons, ppv comes in only a little scrambled! --> syn o n lo dl  o   aetfroi  u r p e en es 
++phronemophilia, spectrophilia, ..... --> uiaw ca io  a a. e  a r  o neo or
in a drawer with winter clothes i was switching stuff over  -->  o e e e   s  n pe  i    r  ete  o flb otiam  i g   a
hope the key lime pie is as good as expected!  --> i    n rorn   e  ag  shdfr  mio   Qen il   w    th
sorry but he sees jimmy then turns, looks at you and says cunt. you then have vagina lips on your face for talking thru -->  o e    o  eoe y i


500
211.724656372 ( 2.18181508416 )	| 2.134 sec
exactly my point, microsoft is doing great in open source let's not mess it up with some random misleading stats --> iaf yein coo reel oe be fhae tume the tolt ah over oos te bod wea i aoiy het ae toinlu. noe.
sorry, katie, we new yorkers aren't giving in to hysteria just yet. you and trump can hold off until we have acts, if you can. --> irs to ieve tot tnsl wor. aaeyt sdllo to iuepl aoes aue ednd s eepm irr irmo oor:rug fy uae boale ttiu lot woet ohmf aafe oo iie attte toae 
to me egyptair was like germanwings in france, a crazed flight crew member who took everyone with him to the end. --> i yhs te aoe telig ohs telto eo datee mord hoem oh sr tnlf whne ae rpanun sekt ere thftle yiese iesr wok diire metntle tee aoesstllroe
i saved that one. --> hoee bho shtrmnrod t to lie chn wlnes 2o fao mvteer mu uotey so
i remember looking down at some and u were playing air guitar during k33p c4lm  --> arrhyn to eo
it's my pleasure. read it, share 


700
200.752402802 ( 2.06874639977 )	| 2.134 sec
*sends u coffee beans, donuts, and gum*  --> lo homte torle bisti ceper iivd to cowlte fau tanoa.
don't break your foot. --> cha wect ife intnes wo lan iout tumele bhat in ii saie lhe eot aopinlnn wis ea ina anmm tvhit tie i mi oa ha
lmfao how u doing brother --> aba wilo i womce anke ta aon os to eee hhaw anu tas ini gfi ia ware pirenor eomt au and aha0.
they also got divorced when i was 10, but you can't prove those things are directly linked --> hao in ai pare do iusn to bout mhn e ea fa wnti or wale at ton anli mone gort au tort kolimie o temti aco oe t aot hactrda waci ios aceid te
nailed that --> wo i dor berten i ave teraal wepyry ihi imbad thte bevt tsoe tor ei anlly tos wus otrtogree tema ii talmsrne an aicet:
shit looks gross --> wit ho phao te inutne ae eae ene moele tnrea ao a lintet anshfrbiudho tce na meee it satsio int fas battte ihmra t lrasie wou ftbinrot one d
save ur talking points. it seems race baiting pays ur bill, 

KeyboardInterrupt: 